In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from refuge.config import load_config
from refuge.training import train

from transformers import Adafactor, GPTNeoXTokenizerFast
from refuge._vendor.mkultra.tuning import GPTNeoXPromptTuningLM

In [ ]:
cfg = load_config()
cfg

In [ ]:
tokenizer = GPTNeoXTokenizerFast.from_pretrained(
    cfg.model.hugging_face_name, padding_side="left"
)
model = GPTNeoXPromptTuningLM.from_pretrained(
    cfg.model.hugging_face_name, device_map="auto"
)

In [ ]:
train(cfg, tokenizer, model)